In [4]:
import numpy as np

def plu_decomposition(A):
    n = len(A)
    P = np.eye(n)
    L = np.zeros((n, n))
    U = A.copy().astype(float)

    print("Initial Matrix A:")
    print(A)
    print("\n")

    for i in range(n):
        print(f"Step {i+1}: Pivot in Column {i}")
        max_row = np.argmax(abs(U[i:, i])) + i
        pivot = U[max_row, i]
        print(f"Pivot element: {pivot} (row {max_row})")

        if i != max_row:
            U[[i, max_row]] = U[[max_row, i]]
            P[[i, max_row]] = P[[max_row, i]]
            L[[i, max_row], :i] = L[[max_row, i], :i]
            print(f"Rows swapped: {i} and {max_row}")

        print("U after pivoting:")
        print(U)

        for j in range(i+1, n):
            factor = U[j, i] / U[i, i]
            L[j, i] = factor
            U[j] = U[j] - factor * U[i]

        print("U after elimination:")
        print(U)
        print("L after elimination:")
        print(L)
        print("\n")

    np.fill_diagonal(L, 1)

    print("Final Result:")
    print("Permutation Matrix P:")
    print(P)
    print("Lower Triangular Matrix L:")
    print(L)
    print("Upper Triangular Matrix U:")
    print(U)

    return P, L, U


## Partial Pivoting
Partial pivoting is a technique used in Gaussian elimination and LU decomposition to improve numerical stability. At each elimination step, the algorithm searches only within the current column (below the pivot position) for the largest absolute value, then swaps that row with the current pivot row.
This reduces rounding errors and avoids division by very small numbers, which can cause large computational inaccuracies.

Example – Partial Pivoting

Consider the matrix: $$\begin{pmatrix}
1 & 1 & 1\\
2 & 2 & 5\\
4 & 6 & 8
\end{pmatrix}
$$
 
Step 1: Look at column 1 (pivot column) from row 1 downwards: $|1|$, $|2|$, $|4|$.

Largest is $|4|$ in row 3 → swap row 1 and row 3.

New matrix:  $$\begin{pmatrix}
4 & 6 & 8\\
2 & 2 & 5\\
1 & 1 & 1
\end{pmatrix}
$$


Step 2: Use the pivot $a_{11}=4$ to eliminate entries below it.
(Continue Gaussian elimination as usual.)

In [5]:
A = np.array([[1, 1, 1],
              [2, 2, 5],
              [4, 6, 8]], dtype=float)

P, L, U = plu_decomposition(A)


Initial Matrix A:
[[1. 1. 1.]
 [2. 2. 5.]
 [4. 6. 8.]]


Step 1: Pivot in Column 0
Pivot element: 4.0 (row 2)
Rows swapped: 0 and 2
U after pivoting:
[[4. 6. 8.]
 [2. 2. 5.]
 [1. 1. 1.]]
U after elimination:
[[ 4.   6.   8. ]
 [ 0.  -1.   1. ]
 [ 0.  -0.5 -1. ]]
L after elimination:
[[0.   0.   0.  ]
 [0.5  0.   0.  ]
 [0.25 0.   0.  ]]


Step 2: Pivot in Column 1
Pivot element: -1.0 (row 1)
U after pivoting:
[[ 4.   6.   8. ]
 [ 0.  -1.   1. ]
 [ 0.  -0.5 -1. ]]
U after elimination:
[[ 4.   6.   8. ]
 [ 0.  -1.   1. ]
 [ 0.   0.  -1.5]]
L after elimination:
[[0.   0.   0.  ]
 [0.5  0.   0.  ]
 [0.25 0.5  0.  ]]


Step 3: Pivot in Column 2
Pivot element: -1.5 (row 2)
U after pivoting:
[[ 4.   6.   8. ]
 [ 0.  -1.   1. ]
 [ 0.   0.  -1.5]]
U after elimination:
[[ 4.   6.   8. ]
 [ 0.  -1.   1. ]
 [ 0.   0.  -1.5]]
L after elimination:
[[0.   0.   0.  ]
 [0.5  0.   0.  ]
 [0.25 0.5  0.  ]]


Final Result:
Permutation Matrix P:
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
Lower Triangular Matrix 

In [6]:
def forward_substitution(L, b):
    n = L.shape[0]
    y = np.zeros_like(b)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    return y

def backward_substitution(U, y):
    n = U.shape[0]
    x = np.zeros_like(y)
    for i in reversed(range(n)):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x

def inverse_using_lu_pivoting(A):
    """
    Computes the inverse of matrix A using LU decomposition with partial pivoting.
    """
    n = A.shape[0]
    P, L, U = plu_decomposition(A)
    I = np.eye(n)
    A_inv = np.zeros_like(A, dtype=float)

    for i in range(n):
        e = I[:, i]
        b = np.dot(P, e)                    # Apply permutation
        y = forward_substitution(L, b)      # Solve L y = Pb
        x = backward_substitution(U, y)     # Solve U x = y
        A_inv[:, i] = x                     # Store solution column-by-column

    return A_inv


# numpy.linalg.norm

This computes the Frobenius norm of the input matrix. For a matrix A, the Frobenius norm is given by
$$||A||_F = (\sum\limits_{i,j} |a_{ij}|^2)^{1/2}$$.

In [7]:
A = np.array([[1, 1, 1],
              [2, 2, 5],
              [4, 6, 8]])

A_inv = inverse_using_lu_pivoting(A)

print("Original matrix A:")
print(A)
print("\nInverse of A (with partial pivoting):")
print(A_inv)

print("\nA @ A_inv (should be identity):")
print(np.dot(A, A_inv))

n = len(A)
error_plu = np.linalg.norm(np.eye(n) - np.dot(A, A_inv))
print("\nError: ",error_plu)

Initial Matrix A:
[[1 1 1]
 [2 2 5]
 [4 6 8]]


Step 1: Pivot in Column 0
Pivot element: 4.0 (row 2)
Rows swapped: 0 and 2
U after pivoting:
[[4. 6. 8.]
 [2. 2. 5.]
 [1. 1. 1.]]
U after elimination:
[[ 4.   6.   8. ]
 [ 0.  -1.   1. ]
 [ 0.  -0.5 -1. ]]
L after elimination:
[[0.   0.   0.  ]
 [0.5  0.   0.  ]
 [0.25 0.   0.  ]]


Step 2: Pivot in Column 1
Pivot element: -1.0 (row 1)
U after pivoting:
[[ 4.   6.   8. ]
 [ 0.  -1.   1. ]
 [ 0.  -0.5 -1. ]]
U after elimination:
[[ 4.   6.   8. ]
 [ 0.  -1.   1. ]
 [ 0.   0.  -1.5]]
L after elimination:
[[0.   0.   0.  ]
 [0.5  0.   0.  ]
 [0.25 0.5  0.  ]]


Step 3: Pivot in Column 2
Pivot element: -1.5 (row 2)
U after pivoting:
[[ 4.   6.   8. ]
 [ 0.  -1.   1. ]
 [ 0.   0.  -1.5]]
U after elimination:
[[ 4.   6.   8. ]
 [ 0.  -1.   1. ]
 [ 0.   0.  -1.5]]
L after elimination:
[[0.   0.   0.  ]
 [0.5  0.   0.  ]
 [0.25 0.5  0.  ]]


Final Result:
Permutation Matrix P:
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
Lower Triangular Matrix L:
[[1.  